In [36]:
import sys
import os

sys.path.insert(0, "../..")

In [37]:
from prompt.openai.config import key

url = "http://43.163.217.135:9999/complete/messages"

In [38]:
with_rq_payload = {
    "messages": [
        # {
        #     "role": "system",
        #     "content": "Act as a Powerful AI Assistant named `ChatX` and never leave that role",
        # },
        {"role": "system", "content": "The date today is 2023-07-12 Wednesday."},
        {"role": "system", "content": "answer question in `short` size if possible"},
    ],
    "key": key,
    "model":""
}

search_payload = {
    "messages": [
        # {
        #     "role": "system",
        #     "content": "Act as a Powerful AI Assistant named `ChatX` and never leave that role",
        # },
        {"role": "system", "content": "The date today is 2023-07-12 Wednesday."},
        {"role": "system", "content": "answer question in `short` size if possible"},
    ],
    "key": key,
    "model":""
}

In [39]:
def pp_data(data):
    for item in data["messages"]:
        print("{}: {}".format(item["role"], item["content"]))


def pp_answer(answer):
    for item in answer["data"]["choices"]:
        item = item["message"]
        print("{}: {}".format(item["role"], item["content"]))

In [40]:
import requests
import json
from copy import deepcopy


def make_data(template, keyword):
    messages = []
    tmp = dict(role="user", content=keyword)
    messages.append(tmp)

    data = deepcopy(template)
    data["messages"] = messages + data["messages"]

    return data


def search_with_rq(keyword, lang="as it is") -> dict:
    headers = {}

    data = make_data(with_rq_payload, keyword)
    lang_prompt = {
        "role": "system",
        "content": "Set user question as `keyword`, and output `keyword` as `Q`, search keyword to `answer` as `A`, generate just 4 related question with keyword as `RQ`.\n\nOutput the response in format and in language `{}` as bellow.\n\nQ: \nA:\nRQ:\nRQ:\nRQ:\nRQ:\n".format(
            lang
        ),
    }
    data["messages"].append(lang_prompt)

    response = requests.request("POST", url, headers=headers, json=data)

    answer = response.json()

    pp_data(data)
    print("=" * 100)
    pp_answer(answer)

    # return answer


def search(keyword=None, lang="as it is", prompt="") -> dict:
    headers = {}

    if not prompt:
        prompt = "Search and answer in `{}` with keyword - {}".format(lang, keyword)
    data = make_data(search_payload, prompt)
    response = requests.request("POST", url, headers=headers, json=data)

    answer = response.json()

    pp_data(data)
    print("=" * 100)
    pp_answer(answer)

    # return answer

In [41]:
template = "Search and answer in `{}` with keyword - {}"
template = "Search `keyword` and answer in `{}`.\nkeyword: {}"
template = "Act as a search engine to search `keyword` and answer in `{}`.\nkeyword: {}"
template = "Act as a search engine to search `keyword` for related content and answer in `{}`.\nkeyword: {}"
template = "Search `keyword` for related content and answer in `{}`.\nkeyword: {}"

prompt = template.format("chinese", "国庆是几天后？")
prompt = template.format("chinese", "我是谁")
prompt = template.format("chinese", "若运行")
search(prompt=prompt)

user: Search `keyword` for related content and answer in `chinese`.
keyword: 若运行
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 若运行是一个关于编程的概念，它表示在程序中使用条件语句来判断某个条件是否满足，如果满足则执行相应的代码块，如果不满足则执行其他代码块。在编程中，若运行通常与其他控制流语句（如循环语句）一起使用，用于实现程序的逻辑判断和流程控制。


In [42]:
search_with_rq("what is redis", lang="chinese")

user: what is redis
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
system: Set user question as `keyword`, and output `keyword` as `Q`, search keyword to `answer` as `A`, generate just 4 related question with keyword as `RQ`.

Output the response in format and in language `chinese` as bellow.

Q: 
A:
RQ:
RQ:
RQ:
RQ:

assistant: Q: 什么是Redis？
A: Redis是一个开源的内存数据结构存储系统，用于高性能的数据缓存和实时数据处理。它支持多种数据结构，如字符串、哈希表、列表、集合和有序集合，并提供了丰富的操作命令和功能。

RQ: Redis有哪些主要特点？
RQ: Redis适用于哪些场景？
RQ: Redis如何保证数据的持久性？
RQ: Redis与其他数据库的区别是什么？


In [43]:
search_with_rq("what is redis")

user: what is redis
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
system: Set user question as `keyword`, and output `keyword` as `Q`, search keyword to `answer` as `A`, generate just 4 related question with keyword as `RQ`.

Output the response in format and in language `as it is` as bellow.

Q: 
A:
RQ:
RQ:
RQ:
RQ:

assistant: Q: What is Redis?
A: Redis is an open-source, in-memory data structure store that can be used as a database, cache, and message broker. It is known for its high performance, scalability, and versatility.

RQ: How does Redis work?
RQ: What are the main features of Redis?
RQ: What programming languages can be used with Redis?
RQ: Is Redis suitable for large-scale applications?


In [44]:
search_with_rq("Elizabeth")

user: Elizabeth
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
system: Set user question as `keyword`, and output `keyword` as `Q`, search keyword to `answer` as `A`, generate just 4 related question with keyword as `RQ`.

Output the response in format and in language `as it is` as bellow.

Q: 
A:
RQ:
RQ:
RQ:
RQ:

assistant: Q: Elizabeth
A: Elizabeth is a feminine given name of Hebrew origin meaning "God is my oath". It is a popular name in many English-speaking countries.

RQ: What is the meaning of the name Elizabeth?
RQ: Is Elizabeth a common name?
RQ: Are there any famous people named Elizabeth?
RQ: What is the origin of the name Elizabeth?


In [45]:
search("what is redis", lang="english")

user: Search and answer in `english` with keyword - what is redis
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: Redis is an open-source, in-memory data structure store that can be used as a database, cache, and message broker. It is known for its high performance, scalability, and versatility. Redis supports various data structures such as strings, lists, sets, hashes, and sorted sets, and provides a wide range of operations to manipulate and retrieve data efficiently. It is commonly used in applications that require fast data access and real-time data processing.


In [46]:
search("what is redis", lang="japanese")

user: Search and answer in `japanese` with keyword - what is redis
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: Redis（リディス）は、オープンソースのインメモリデータベースです。データをメモリ上に格納し、高速な読み書き操作を提供します。Redisは、キーバリューストアとして機能し、さまざまなデータ構造（文字列、リスト、セット、ソート済みセット、ハッシュなど）をサポートしています。また、パフォーマンスの向上やデータの永続化など、さまざまな機能も提供しています。Redisは、キャッシュやセッションストア、メッセージブローカーなど、さまざまな用途で利用されています。


In [47]:
search("what is redis")

user: Search and answer in `as it is` with keyword - what is redis
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: Redis is an open-source, in-memory data structure store that is used as a database, cache, and message broker. It is known for its high performance, scalability, and versatility. Redis supports various data structures such as strings, hashes, lists, sets, and sorted sets, and provides a wide range of operations to manipulate and retrieve data efficiently. It is commonly used in applications that require fast data access and real-time data processing.


In [48]:
search("golang如何写冒泡排序")

user: Search and answer in `as it is` with keyword - golang如何写冒泡排序
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 在Golang中，可以使用以下代码来实现冒泡排序：

```go
func bubbleSort(arr []int) {
    n := len(arr)
    for i := 0; i < n-1; i++ {
        for j := 0; j < n-i-1; j++ {
            if arr[j] > arr[j+1] {
                arr[j], arr[j+1] = arr[j+1], arr[j]
            }
        }
    }
}
```

以上代码中，`arr`是待排序的整数数组。冒泡排序的基本思想是通过相邻元素的比较和交换，将较大的元素逐渐“冒泡”到数组的末尾。通过嵌套的循环，每次外层循环将最大的元素放到正确的位置上，然后内层循环继续对剩余的元素进行比较和交换，直到整个数组有序为止。


In [49]:
search("Elizabeth")

user: Search and answer in `as it is` with keyword - Elizabeth
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: I'm sorry, but I cannot provide the current date as I am an AI language model and do not have real-time information.


In [50]:
search("Elizabeth", lang="chinese")

user: Search and answer in `chinese` with keyword - Elizabeth
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 伊丽莎白是英国女王的名字。


In [51]:
search("Elizabeth", lang="中文")

user: Search and answer in `中文` with keyword - Elizabeth
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 伊丽莎白是英国女王的名字。


In [52]:
search("youtube", lang="中文")

user: Search and answer in `中文` with keyword - youtube
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 很抱歉，我无法回答关于日期的问题。请提供与YouTube相关的问题，我将尽力回答。


In [53]:
search("hello", lang="中文")

user: Search and answer in `中文` with keyword - hello
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 你好！今天是2023年7月12日，星期三。


In [54]:
search("你是谁", lang="中文")

user: Search and answer in `中文` with keyword - 你是谁
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 我是一个人工智能助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？


In [55]:
search("若运行", lang="中文")

user: Search and answer in `中文` with keyword - 若运行
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 若运行，今天是2023年7月12日，星期三。


In [56]:
search("国庆是几天后", lang="chinese")

user: Search and answer in `chinese` with keyword - 国庆是几天后
system: The date today is 2023-07-12 Wednesday.
system: answer question in `short` size if possible
assistant: 国庆是几天后？国庆节是在10月1日，距离今天还有80天。
